In [1]:
from molecular_mpns.config import data_dir
import numpy as np 
import mdtraj as md
import os
from torch_geometric.nn.models.dimenet import DimeNet
from molecular_mpns.data import AlanineDipeptideGraph
from torch_geometric.data import DataLoader
import torch

In [2]:
# load training trajectory
os.chdir(data_dir)
os.chdir('alanine_dipeptide/')
xtc_file = 'alanine-dipeptide-0-250ns-nowater.xtc'
top_file = 'alanine-dipeptide-nowater.pdb'
traj = md.load(xtc_file,top=top_file)

# create graphs
batch_size = 32
z = [atom.element.atomic_number for atom in traj.topology.atoms]
G = [AlanineDipeptideGraph(z = torch.tensor(z).long(),pos = torch.tensor(xyz)) for xyz in traj.xyz]
loader = DataLoader(G,batch_size = batch_size)

In [3]:
# build dimenet model
hidden_channels, out_channels, num_blocks, num_bilinear, num_spherical, num_radial = 64, 16, 2, 4, 6, 6
mod = DimeNet(hidden_channels, out_channels, num_blocks, num_bilinear, num_spherical, num_radial)

for G_batch in loader:
    batch = G_batch.batch
    out = mod(G_batch.z,G_batch.pos,batch)
    break